In [1]:
#Importing all the neccessary libraries
import pandas as pd
import numpy as np #for linear algebra

#Libraries for visualisation
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import seaborn as sns
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go

#Libraries for formattting and handling text 
import string
import re

#Library for nltk
import nltk 
import nltk.corpus
from nltk.corpus import stopwords
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('wordnet2022')
nltk.download('stopwords')
! cp -rf /usr/share/nltk_data/corpora/wordnet2022 /usr/share/nltk_data/corpora/wordnet
from nltk.corpus import wordnet
from nltk.stem import WordNetLemmatizer
from wordcloud import WordCloud
from nltk.sentiment import SentimentIntensityAnalyzer

from collections import defaultdict
from collections import  Counter
stop=set(stopwords.words('english'))

#Library for NN
from keras.models import Sequential
from keras.layers import Embedding, LSTM, Dense, SpatialDropout1D
from keras.initializers import Constant


import sklearn
#Library for Splitting Dataset
from sklearn.model_selection import train_test_split
#Library for TFIDF
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer, HashingVectorizer
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn import model_selection, naive_bayes
#Library for evaluation
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score,classification_report

import warnings
warnings.filterwarnings("ignore")

import plotly
plotly.offline.init_notebook_mode (connected = True)

!pip install neattext
import neattext as nt

# Loading Dependencies
import os
import tensorflow as tf
from tensorflow.keras.layers import Dense, Input
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import ModelCheckpoint
import transformers

from tokenizers import BertWordPieceTokenizer
from transformers import AutoTokenizer, AutoModel

import torch

from sklearn.decomposition import PCA

[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /usr/share/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package wordnet2022 to /usr/share/nltk_data...
[nltk_data]   Unzipping corpora/wordnet2022.zip.
[nltk_data] Downloading package stopwords to /usr/share/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


/opt/conda/lib/python3.7/site-packages/nltk/twitter/__init__.py:20: UserWarning: The twython library has not been installed. Some functionality from the twitter package will not be available.
  warnings.warn("The twython library has not been installed. "


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.7/114.7 kB 3.3 MB/s eta 0:00:00


In [2]:
def clean_data(text):
    #pipeline cleans in order, normalize first, then remove_numbers ETC
    clean_pipeline = nt.TextPipeline(steps=[nt.normalize, nt.remove_numbers, nt.fix_contractions, nt.remove_stopwords, nt.remove_emails, nt.remove_emojis, nt.remove_html_tags, nt.remove_urls, nt.remove_special_characters])
    text = clean_pipeline.transform(text)
 
    #Lemmatization -> reducing a word to its base or dictionary form: "am", "is", "are" -become-> "be"
        # this will normalize the text data and reduce its dimensionality (amount of data)
    lemmatizer = WordNetLemmatizer()
    text = " ".join([lemmatizer.lemmatize(word) for word in text.split()])
    text = " ".join([t for t in text.split() if len(t) > 3 ])
    return text

In [3]:
df = pd.read_csv("/kaggle/input/nlp-getting-started/train.csv")
df['text'] = df['text'].apply(clean_data)

tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')
model = AutoModel.from_pretrained('bert-base-uncased')
df.head(5)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


,id,keyword,location,text,target
0,1,NaN,NaN,deed reason earthquake allah forgive,1
1,4,NaN,NaN,forest fire near ronge sask canada,1
2,5,NaN,NaN,resident asked shelter place notified officer ...,1
3,6,NaN,NaN,people receive wildfire evacuation order calif...,1
4,7,NaN,NaN,sent photo ruby alaska smoke wildfire pours sc...,1


In [4]:
def model_exec(x_train, y_train, x_val, y_val, models=None):
    if models is None:
        models = [
              DecisionTreeClassifier(),
              GradientBoostingClassifier(),
              sklearn.naive_bayes.MultinomialNB(), 
              sklearn.linear_model.LogisticRegression(),
              sklearn.svm.SVC(),
              sklearn.svm.SVC(C=10),
              sklearn.svm.SVC(C=0.5),
            ]
    else:
        if not isinstance(models, list):
            models = list(models)
            
    accuracy_models = []
    for model in models:
        model.fit(x_train, y_train)
        predictions = model.predict(x_val)
        print("Accuracy Score -> ", model , accuracy_score(predictions, y_val)*100)
        accuracy_models.append((accuracy_score(predictions, y_val)*100, model))
        
    print(accuracy_models)
    return max(accuracy_models)

In [5]:
# Define the function to process the text data
def process_text(text_data):
    # Tokenize the text data using the BERT tokenizer
    tokenized_text = text_data.apply((lambda x: tokenizer.encode(x, add_special_tokens=True)))

    # Pad the tokenized sequences to ensure that they are all the same length
    max_len = 50 # set the maximum length of the input sequence
    padded_text = np.array([i + [0]*(max_len-len(i)) for i in tokenized_text.values])

    # Convert the input data to PyTorch tensors
    input_ids = torch.tensor(padded_text)  

    # Pass the input data through the BERT model
    with torch.no_grad():
        last_hidden_states = model(input_ids)

    # Extract the last hidden state of the first token
    features = last_hidden_states[0][:,0,:].numpy()

    return features

In [6]:
# Feature from BERT
features = process_text(df['text'])
#TF-IDF features
vectorizer = TfidfVectorizer()
tfidf_features = vectorizer.fit_transform(df['text'])

# Concatenate the BERT features and TF-IDF features
features = np.concatenate((features, tfidf_features.toarray()), axis=1)
features[:1]

array([[-0.42209759,  0.32885087,  0.46404377, ...,  0.        ,
         0.        ,  0.        ]])

In [7]:
# Split the Bert/TFIDF features into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(features, df["target"], test_size=0.2, random_state=42)
type(X_train)

numpy.ndarray

In [8]:
#find the best model
model_exec(X_train, y_train, X_test, y_test,[
              DecisionTreeClassifier(),
              GradientBoostingClassifier(),
              #sklearn.naive_bayes.MultinomialNB(), 
              sklearn.linear_model.LogisticRegression(),
              sklearn.svm.SVC(),
              sklearn.svm.SVC(C=10),
              sklearn.svm.SVC(C=0.5),
            ])

Accuracy Score ->  DecisionTreeClassifier() 65.52856204858831
Accuracy Score ->  GradientBoostingClassifier() 79.25147734734077
Accuracy Score ->  LogisticRegression() 79.51411687458962
Accuracy Score ->  SVC() 78.59487852921865
Accuracy Score ->  SVC(C=10) 80.69599474720945
Accuracy Score ->  SVC(C=0.5) 77.74130006565989
[(65.52856204858831, DecisionTreeClassifier()), (79.25147734734077, GradientBoostingClassifier()), (79.51411687458962, LogisticRegression()), (78.59487852921865, SVC()), (80.69599474720945, SVC(C=10)), (77.74130006565989, SVC(C=0.5))]


(80.69599474720945, SVC(C=10))